In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
con = sqlite3.connect(r'C:\Users\krush\Desktop\Data Science\SQL/Db-IMDB-Assignment.db')

                                                --- Question 1 ---
List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year. 

In [6]:
ques1 = """ SELECT  p.name as director_name, m.title as movie_name, SUBSTRING(m.year, -4, 4) as movie_year from
    Movie m JOIN M_Director md ON m.MID = md.MID
    JOIN Person p ON p.PID = md.PID 
    JOIN M_Genre mg ON mg.MID = md.MID
    JOIN Genre g ON g.GID = mg.GID
    where g.Name Like '%Comedy%' 
    AND (
    (TRIM(m.year) % 4 = 0 AND TRIM(m.year) % 100 != 0 ) 
    OR ( TRIM(m.year) % 400 = 0) )"""
ques1conn = pd.read_sql_query(ques1,con)
print(ques1conn)

              director_name                         movie_name movie_year
0              Milap Zaveri                         Mastizaade       2016
1              Danny Leiner  Harold & Kumar Go to White Castle       2004
2            Anurag Kashyap                 Gangs of Wasseypur       2012
3              Frank Coraci        Around the World in 80 Days       2004
4             Griffin Dunne             The Accidental Husband       2008
..                      ...                                ...        ...
241   Siddharth Anand Kumar                        Let's Enjoy       2004
242         Amma Rajasekhar                            Sathyam       2008
243           Oliver Paulus                      Tandoori Love       2008
244             Raja Chanda                        Le Halua Le       2012
245        K.S. Prakash Rao                  Raja Aur Rangeeli       1996

[246 rows x 3 columns]


                                                --- Question 2 ---
Question 2 : List the names of all the actors who played in the movie 'Anand' (1971) 

In [7]:
ques2 = """SELECT p.Name as Actor_Name FROM Person p WHERE p.PID IN
           (SELECT TRIM(mc.PID) FROM M_Cast mc Where mc.MID IN
           (SELECT TRIM(MID) FROM Movie m where m.Title Like '%Anand%' and m.year = '1971'))"""
ques2conn = pd.read_sql_query(ques2,con)
print(ques2conn)

           Actor_Name
0    Amitabh Bachchan
1       Rajesh Khanna
2       Sumita Sanyal
3          Ramesh Deo
4           Seema Deo
5      Asit Kumar Sen
6          Dev Kishan
7        Atam Prakash
8       Lalita Kumari
9              Savita
10     Brahm Bhardwaj
11       Gurnam Singh
12       Lalita Pawar
13        Durga Khote
14         Dara Singh
15      Johnny Walker
16          Moolchand


                                                 --- Question 3 ---
Question 3: List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 
and > 1990.) 
                                              

In [8]:
ques3 = """SELECT p.Name FROM Person p WHERE PID IN(
          SELECT TRIM(mc.PID) FROM M_Cast mc WHERE MID IN
          (SELECT TRIM(m.MID) FROM Movie m where m.year > 1990)
          INTERSECT
          SELECT TRIM(mc.PID) FROM M_Cast mc WHERE MID IN
          (SELECT TRIM(m.MID) FROM Movie m where m.year < 1970))
"""
ques3con = pd.read_sql_query(ques3,con)
print(ques3con)

                  Name
0         Rishi Kapoor
1     Amitabh Bachchan
2               Asrani
3         Zohra Sehgal
4      Parikshat Sahni
..                 ...
328             Poonam
329      Jamila Massey
330        K.R. Vijaya
331              Sethi
332       Suryakantham

[333 rows x 1 columns]


                                                 --- Question 4 ---
Question 4: List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed. 


In [9]:
ques4 = """SELECT p.Name as Director_Name, COUNT(p.Name) FROM Movie m JOIN 
           M_Director md ON md.MID = m.MID JOIN
           Person p ON p.PID = md.PID GROUP BY p.Name HAVING COUNT(p.Name) > 10 ORDER BY COUNT(p.Name) DESC
"""
ques4con =pd.read_sql_query(ques4,con)
print(ques4con)

                Director_Name  COUNT(p.Name)
0                David Dhawan             39
1                Mahesh Bhatt             36
2                Priyadarshan             30
3             Ram Gopal Varma             30
4                Vikram Bhatt             29
5        Hrishikesh Mukherjee             27
6                 Yash Chopra             21
7              Shakti Samanta             19
8             Basu Chatterjee             19
9                Subhash Ghai             18
10          Rama Rao Tatineni             17
11              Shyam Benegal             17
12   Abbas Alibhai Burmawalla             17
13               Raj N. Sippy             16
14             Manmohan Desai             16
15                     Gulzar             16
16                 Raj Kanwar             15
17           Mahesh Manjrekar             15
18          Rajkumar Santoshi             14
19                 Raj Khosla             14
20               Rahul Rawail             14
21        

                                                 --- Question 5 ---
Question 5: A. For each year, count the number of movies in that year that had only female actors. 
B. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer. 


In [29]:
ques5A = """SELECT COUNT(TRIM(MID)) as female_mcount, m.year as Year FROM Movie m WHERE TRIM(MID) NOT IN
            (SELECT DISTINCT  TRIM(mc.MID) FROM M_Cast mc WHERE TRIM(PID) IN
            (SELECT p.PID FROM Person p  WHERE TRIM(p.Gender) = 'Male'))  GROUP BY m.year 
            

"""
ques5Aconn = pd.read_sql_query(ques5A,con)
print(ques5Aconn)

   female_mcount    Year
0              1    1939
1              1    1999
2              1    2000
3              1  I 2018


In [18]:
ques5B = """SELECT m.year as Year, COUNT(m.MID) as Total_Movie_Count,female_mcount, (female_mcount * 100.0 / COUNT(m.MID)) as Percentage FROM Movie m JOIN
            (SELECT COUNT(TRIM(MID)) as female_mcount, m.year as Year FROM Movie m WHERE TRIM(MID) NOT IN
            (SELECT DISTINCT  TRIM(mc.MID) FROM M_Cast mc WHERE TRIM(PID) IN
            (SELECT p.PID FROM Person p  WHERE TRIM(p.Gender) = 'Male'))  GROUP BY m.year)as fmovies_count ON  m.year = fmovies_count.year GROUP BY m.year
            
            

"""
ques5Bconn = pd.read_sql_query(ques5B,con)
print(ques5Bconn)

     Year  Total_Movie_Count  female_mcount  Percentage
0    1939                  2              1   50.000000
1    1999                 66              1    1.515152
2    2000                 64              1    1.562500
3  I 2018                 10              1   10.000000


                                                 --- Question 6 ---
Question 6:. Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once. 

In [37]:
ques6 = """SELECT r.Title as Movie_Title, MAX(Cast_Count) as Max_Cast_Count FROM 
            (SELECT COUNT(DISTINCT mc.PID) as Cast_Count, m.Title FROM Movie m
           JOIN M_Cast mc ON m.MID = mc.MID GROUP BY m.MID ORDER BY Cast_Count DESC) as r"""
ques6conn = pd.read_sql_query(ques6,con)
print(ques6conn)

     Movie_Title  Max_Cast_Count
0  Ocean's Eight             238


                                                 --- Question 7 ---
Question 7:A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.  

In [105]:
ques7  = """WITH 
            UNIQUE_YEAR AS (SELECT DISTINCT "year" FROM Movie)

            SELECT D."year" AS START, D."year"+9 AS END, COUNT(*) AS 'COUNT' 
            FROM UNIQUE_YEAR D
            JOIN Movie M on M."year" >= START AND M."year"<= END
            GROUP BY END 
            ORDER BY COUNT(*) DESC 
            LIMIT 1 """
ques7conn = pd.read_sql_query(ques7,con)
print(ques7conn)

  START   END  COUNT
0  2008  2017   1126


                                              --- Question 8 ---
Question 8:Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies). 
 

In [35]:
ques1 = """ WITH Movie_Year AS (
        SELECT DISTINCT TRIM(MC.PID) AS PID, TRIM(M.Year) AS YEAR, 
        ROW_NUMBER() OVER (PARTITION BY TRIM(MC.PID) ORDER BY Year) Row_Num 
        FROM Movie M JOIN M_Cast MC ON TRIM(M.MID)=TRIM(MC.MID) 
         )

        SELECT DISTINCT Name FROM Person WHERE PID NOT IN (
        SELECT DISTINCT M1.PID FROM Movie_Year M1 JOIN Movie_Year M2 
        ON M1.PID=M2.PID AND M1.Row_Num+1=M2.Row_Num
        WHERE M2.Year-M1.Year >= 3  ) """
ques1conn = pd.read_sql_query(ques1,con)
print(ques1conn)

                        Name
0             Christian Bale
1             Cate Blanchett
2       Benedict Cumberbatch
3              Naomie Harris
4                Andy Serkis
...                      ...
31090           Kamika Verma
31091     Dhorairaj Bhagavan
31092           Nasir Shaikh
31093                 Kannan
31094           Adrian Fulle

[31095 rows x 1 columns]


                                            --- Question 9 ---
Question 9:Find all the actors that made more movies with Yash Chopra than any other.  
 

In [4]:
ques9 = """  SELECT p.Name, list.Count FROM Person p JOIN 
            (SELECT mc.PID,COUNT(mc.PID) as Count FROM M_Cast mc WHERE TRIM(MID) In
            (SELECT md.MID FROM M_Director md WHERE TRIM(PID) IN
            (SELECT p.PID FROM Person p WHERE TRIM(p.NAME) = 'Yash Chopra')) GROUP BY mc.PID )as list ON TRIM(p.PID) = TRIM(list.PID) ORDER BY list.count DESC"""
ques9conn = pd.read_sql_query(ques9,con)
print(ques9conn)

                  Name  Count
0          Jagdish Raj     11
1     Manmohan Krishna     10
2             Iftekhar      9
3          Vikas Anand      8
4           Madan Puri      8
..                 ...    ...
425         Ramchandra      1
426    Sandow S. Sethi      1
427              Naval      1
428          Prem Sood      1
429    Ramlal Shyamlal      1

[430 rows x 2 columns]


                                                 --- Question 10 ---
Question 10:Find all the actors that made more movies with Yash Chopra than any other.  

In [34]:
ques10 = """ SELECT  TRIM(p.NAME) as Actor_Name FROM Person p WHERE TRIM(PID) IN
            (SELECT TRIM(mc.PID) as S2ID FROM M_Cast mc WHERE TRIM(MID) IN
            (SELECT TRIM(mc.MID) FROM M_cast mc WHERE TRIM(PID) IN
            (SELECT TRIM(mc.PID) FROM M_Cast mc WHERE TRIM(MID) IN
            (SELECT TRIM(mc.MID) FROM M_Cast mc WHERE TRIM(PID) IN
            (SELECT TRIM(p.PID) FROM Person p WHERE TRIM(p.NAME) ='Shah Rukh Khan'))))
            EXCEPT
            SELECT TRIM(mc.PID) FROM M_Cast mc WHERE TRIM(mc.MID) IN
            (SELECT TRIM(mc.MID) FROM M_Cast mc WHERE TRIM(PID) IN
            (SELECT TRIM(p.PID) FROM Person p WHERE TRIM(p.NAME) ='Shah Rukh Khan')))
            """


ques10conn = pd.read_sql_query(ques10,con)
print(ques10conn)

                  Actor_Name
0               Freida Pinto
1                Rohan Chand
2               Damian Young
3            Waris Ahluwalia
4      Caroline Christl Long
...                      ...
25693           Minoo Mehtab
25694        Hayley Cleghorn
25695        Nirvasha Jithoo
25696         Kamal Maharshi
25697           Mohini Manik

[25698 rows x 1 columns]
